In [1]:
# %matplotlib qt
from scipy.io import loadmat
import numpy as np
import pandas as pd
import mat73
import mne
import os

import inspect #path_fun = inspect.getfile(StratifiedKFold)
import matplotlib.pyplot as plt
import DecToolbox as dt
#print(sys.path)

import sys
path_utils = '/decoding_toolbox_py/helper_funcs' 
sys.path.append(path_utils)
# !pip install mat73

In [2]:
'''
GLOBAL VARIABLES
'''

read_EGG = True

ival = 0

In [17]:
if read_EGG:
    subjs_list = ['s01', 's02'
              #, 's03' ,'s04','s05','s07','s08','s09','s10','s11','s12','s13','s14', 's15', 's16','s17','s18','s19','s20','s21','s22','s23','s24','s25','s26','s27' 
                ]
    path = 'Cond_CJ_EEG'

    epochs = []
    for isubj, subject_id in enumerate(subjs_list):
        preproc_path = os.path.join(path, subject_id);
        epoch = mne.read_epochs(preproc_path + '/main_epo.fif',verbose=False)
        epochs.append(epoch.average());
        # print(isubj)

    gvaverage = mne.grand_average(epochs[:])
    all_epochs = []
    all_rawdata = [None] * np.size(subjs_list)
    for isubj, subject_id in enumerate(subjs_list):
        preproc_path = os.path.join(path, subject_id);
        epoch = mne.read_epochs(preproc_path + '/main_epo.fif',verbose=False)
        subj_data = {'epoch_dat' : epoch.get_data(), 'metadata' :  epoch.metadata}
        all_rawdata[isubj] =  subj_data
        all_epochs.append(epoch);
        epoch.metadata
        
    all_st_epochs = []
    all_st_rawdata = [None] * np.size(subjs_list)
    for isubj, subject_id in enumerate(subjs_list):
        preproc_path = os.path.join(path, subject_id);
        epoch = mne.read_epochs(preproc_path + '/mainstim_epo.fif',verbose=False)
        subj_data = {'epoch_dat' : epoch.get_data(), 'metadata' :  epoch.metadata}
        all_st_rawdata[isubj] =  subj_data
        all_st_epochs.append(epoch);
        epoch.metadata

    tt = all_st_epochs[ival].times
    ll= all_st_epochs[ival].ch_names
    YY = all_st_rawdata[ival]['epoch_dat']
    YY = np.einsum('kji->jik', YY)
    XX = np.array(all_st_rawdata[ival]['metadata'].orient)
    binned_XX = (np.digitize(XX, bins = np.array([  0, 22.5, 45., 67.5, 90, 112.5, 135, 157.5 ]) ) -1).astype(float)# bins collapsing degrees by similarity 


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Identifying common channels ...


In [18]:
print(YY.shape)
print(XX.shape)

(32, 201, 1563)
(1563,)


In [19]:
path = 'decoding-toolbox_matlab/data/testdata_orientation.mat' 
ori_data= mat73.loadmat(path)

In [20]:
'''Matlab'''
Y = ori_data['Y']
X = ori_data['X']
time = ori_data['time']
label = ori_data['label'] # Label of the features
print('Y', Y.shape,'X', X.shape,'time', time.shape)

Y (273, 211, 760) X (760,) time (211,)


In [21]:
np.unique(binned_XX)


array([0., 1., 2., 3., 4., 5., 6., 7.])

In [22]:
np.unique(X)

array([0., 1., 2., 3., 4., 5., 6., 7.])